In [5]:
# 忽略各模块的警告信息
import warnings
warnings.filterwarnings("ignore")

In [6]:
# 加载所需使用的模块
from datetime import datetime

from vnpy.trader.datafeed import get_datafeed
from vnpy.trader.database import get_database, DB_TZ
from vnpy.trader.constant import Interval
from vnpy.trader.object import BarData, HistoryRequest
from vnpy.trader.utility import extract_vt_symbol
from vnpy.trader.setting import SETTINGS
from akshare import stock_feature as sf

# 配置数据服务
SETTINGS["datafeed.name"] = "akshare"            # 可以根据自己的需求选择数据服务：rqdata/xt/wind等
SETTINGS["datafeed.username"] = "license"       # RQData的用户名统一为“license”这个字符串
SETTINGS["datafeed.password"] = "123456"        # 这里需要替换为你购买或者申请试用的RQData数据license

# 配置数据库
SETTINGS["database.name"] = "taos"              # 可以根据自己的需求选择数据库，这里使用的是TDengine
SETTINGS["database.database"] = "vnpy"
SETTINGS["database.host"] = "127.0.0.1"
SETTINGS["database.port"] = 6030
SETTINGS["database.user"] = "root"
SETTINGS["database.password"] = "taosdata"

# 创建对象实例
datafeed = get_datafeed()

# database = get_database()

# 要下载数据的起止时间
start = datetime(2018, 5, 30, tzinfo=DB_TZ)
end = datetime(2025, 7, 4, tzinfo=DB_TZ)

In [7]:
# 要下载数据的合约代码
vt_symbols = [
    "000001.SZSE",

    # "IF2501.CFFEX",

    # "IH2501.CFFEX",

    # "IC2501.CFFEX",

    # "IM2501.CFFEX",
]

In [8]:
# 遍历列表执行下载
for vt_symbol in vt_symbols:
    # 拆分合约代码和交易所
    symbol, exchange = extract_vt_symbol(vt_symbol)

    # 创建历史数据请求对象
    req: HistoryRequest = HistoryRequest(
        symbol=symbol,
        exchange=exchange,
        start=start,
        end=end,
        interval=Interval.DAILY
    )

    # 从数据服务下载数据
    bars: list[BarData] = datafeed.query_bar_history(req)

    if bars:
        print(bars)
        # database.save_bar_data(bars)
        print(f"下载数据成功：{vt_symbol}，总数据量：{len(bars)}")
    else:
        print(f"下载数据失败：{vt_symbol}")

[BarData(gateway_name='AK', extra=None, symbol='000001', exchange=<Exchange.SZSE: 'SZSE'>, datetime=datetime.datetime(2018, 5, 30, 15, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>), interval=<Interval.DAILY: 'd'>, volume=1124447, turnover=1139918752.0, open_interest=0, open_price=7.77, high_price=7.77, low_price=7.53, close_price=7.56), BarData(gateway_name='AK', extra=None, symbol='000001', exchange=<Exchange.SZSE: 'SZSE'>, datetime=datetime.datetime(2018, 5, 31, 15, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>), interval=<Interval.DAILY: 'd'>, volume=987360, turnover=999216336.0, open_interest=0, open_price=7.59, high_price=7.67, low_price=7.5, close_price=7.66), BarData(gateway_name='AK', extra=None, symbol='000001', exchange=<Exchange.SZSE: 'SZSE'>, datetime=datetime.datetime(2018, 6, 1, 15, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>), interval=<Interval.DAILY: 'd'>, volume=597211, turnover=608398864.0, open_interest=0, open_price=7.63, high_price=7.77,